In [3]:
import numpy as np


# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


# sigmoid 함수
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [4]:
# LogicGate Class

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata, lr, cnt):  # xdata, tdata => numpy.array(...)
        
        self.name = gate_name
        
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 가중치 W, 바이어스 b 초기화 - 일반적인 방법으로 변경 필요
        self.W = np.random.rand(self.xdata.shape[1], 1)  # weight, 2 X 1 matrix
        self.b = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = lr
        
        # 반복횟수 초기화
        self.iteration_count = cnt
        
    # 손실함수
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )      
    
    # 손실 값 계산
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        step_count = int(self.iteration_count * 0.1)
        
        f = lambda x : self.loss_func()
        
        print("Initial error value = ", self.error_val())
        
        for step in  range(self.iteration_count):
            
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % step_count == 0):
                print("step = ", step, "error value = ", self.error_val())
                
                
    # 미래 값 예측 함수
    def predict(self, input_data):
        
        z = np.dot(input_data, self.W) + self.b
        y = sigmoid(z)
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy(self, test_xdata, test_tdata):
        
        matched_list = []
        
        for index in range(len(xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
                
        return ( len(matched_list) / len(test_xdata))
    
    # 인덱스 주소값만 호출하여 정답과 문제를 비교함

In [5]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

AND_obj = LogicGate("AND_GATE", xdata, tdata, 1e-2, 20001)

AND_obj.train()

Initial error value =  4.146818333285336
step =  0 error value =  4.097847657226809
step =  2000 error value =  0.6634829610647681
step =  4000 error value =  0.3914225699594567
step =  6000 error value =  0.27548788398524604
step =  8000 error value =  0.21160292873480674
step =  10000 error value =  0.1713481474173434
step =  12000 error value =  0.1437497817339244
step =  14000 error value =  0.12369225393143701
step =  16000 error value =  0.1084780248486814
step =  18000 error value =  0.09655363108579801
step =  20000 error value =  0.08696299316026268


In [6]:
# AND Gate prediction
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

accuracy_ret = AND_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

AND_GATE 

Accuracy =>  1.0


In [5]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata, 1e-2, 20001)

OR_obj.train() 

Initial error value =  1.8106557906089031
step =  0 error value =  1.8068450163125744
step =  2000 error value =  0.42543372135233737
step =  4000 error value =  0.23063735398879856
step =  6000 error value =  0.15617844728720348
step =  8000 error value =  0.11749495570516137
step =  10000 error value =  0.09395636998639567
step =  12000 error value =  0.07817801055697159
step =  14000 error value =  0.06688709219926361
step =  16000 error value =  0.058417727850586146
step =  18000 error value =  0.05183498501704856
step =  20000 error value =  0.046574566323333545


In [6]:
# OR Gate prediction
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

accuracy_ret = OR_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

OR_GATE 

Accuracy =>  1.0


In [7]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata, 1e-2, 20001)

NAND_obj.train()

Initial error value =  2.8890862389450107
step =  0 error value =  2.880681567896533
step =  2000 error value =  0.6773289531475155
step =  4000 error value =  0.3963192001207009
step =  6000 error value =  0.27796184446587924
step =  8000 error value =  0.2130826904054869
step =  10000 error value =  0.17232777771302304
step =  12000 error value =  0.1444440547682529
step =  14000 error value =  0.12420899031436375
step =  16000 error value =  0.10887707752685463
step =  18000 error value =  0.09687079955086628
step =  20000 error value =  0.08722096237210004


In [8]:
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

accuracy_ret = NAND_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

NAND_GATE 

Accuracy =>  1.0


In [11]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


XOR_obj = LogicGate("XOR_GATE", xdata, tdata, 1e-1, 200001)

XOR_obj.train()

# 정확도가 낮다: 분류하려면 최소 직선이 2개가 필요하다
# logestic으로는 안풀린다
# 마빈 민스키 xor problem

Initial error value =  3.1621457466800695
step =  0 error value =  3.075147929493262
step =  20000 error value =  2.7725879222398615
step =  40000 error value =  2.7725879222398615
step =  60000 error value =  2.7725879222398615
step =  80000 error value =  2.7725879222398615
step =  100000 error value =  2.7725879222398615
step =  120000 error value =  2.7725879222398615
step =  140000 error value =  2.7725879222398615
step =  160000 error value =  2.7725879222398615
step =  180000 error value =  2.7725879222398615
step =  200000 error value =  2.7725879222398615


In [16]:
# XOR Gate prediction
print(XOR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

accuracy_ret = XOR_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

XOR_GATE 

Accuracy =>  0.25
